# Weekly Report for Hospital Data

In [24]:
# Imports
import sys
import pandas as pd
import psycopg
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import numpy as np

from credentials import DB_PASSWORD, DB_USER

In [25]:
# Parameters

week = "2022-9-30"

In [26]:
# Connect to DB
conn = psycopg.connect(
    host="sculptor.stat.cmu.edu", dbname=DB_USER,
    user=DB_USER, password=DB_PASSWORD
)
cur = conn.cursor()

In [27]:
# Summary I & II
with conn.transaction():
    cur.execute(
        "SELECT report_date AS week, COUNT(*) AS records_loaded, "
        "SUM(total_adult_hospital_beds), SUM(total_pediatric_hospital_beds), "
        "SUM(total_adult_hospital_beds_occupied), "
        "SUM(total_pediatric_hospital_beds_occupied), "
        "SUM(inpatient_beds_occupied_covid) "
        "FROM facility_reports "
        "GROUP BY report_date "
        "ORDER BY week DESC;"
    )

records = pd.DataFrame(
    cur.fetchall(), columns=["Week", "Records Loaded", "Total Adult Hospital Beds Available", 
                             "Total Pediatric Hospital Beds Available", 
                             "Total Adult Hospital Beds In-Use", "Total Pediatric Hospital Beds In-Use",
                             "Total Hospital Beds Used by COVID cases"]
)

In [28]:
# I table
import plotly.figure_factory as ff

colorscale1 = [[0, '#379392'],[.5, '#9DC3C1'],[1, '#D8E6E7']]
colorscale2 = [[0, '#eb9f9f'],[.5, '#f1bbba'],[1, '#FADAD8']]
colorscale3 = [[0, '#79bd9a'],[.5, '#a8dba8'],[1, '#cff09e']]
colorscale4 = [[0, '#6a60a9'],[.5, '#dedcee'],[1, '#F8FAFF']]
df_1 = records[["Week", "Records Loaded"]]

fig_1 = ff.create_table(df_1, colorscale=colorscale4)

fig_1.update_layout(
    {'margin':{'t':30}},
    title={
        "text":"Summary of loaded records",
        "y":0.95,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    autosize=False,
    width=150*df_1.shape[1],
    height=40*df_1.shape[0]
)

fig_1.show()

In [29]:
# II
records

,Week,Records Loaded,Total Adult Hospital Beds Available,Total Pediatric Hospital Beds Available,Total Adult Hospital Beds In-Use,Total Pediatric Hospital Beds In-Use,Total Hospital Beds Used by COVID cases
0,2022-10-21,4985,414880.3,39882.2,469153.4,29706.5,24116.8
1,2022-10-14,4995,412939.0,39711.6,467145.9,28799.0,23695.5
2,2022-10-07,4996,415465.4,39796.8,465980.5,28285.8,23689.3
3,2022-09-30,4997,416279.4,39804.7,462204.5,28042.8,24142.5
4,2022-09-23,4996,413589.5,39605.5,463698.4,27778.1,25600.6


In [63]:
# I table
df_2 = records
df_2.columns = ["Week", "Records Loaded", "Total Adult Hospital<br />Beds Available", 
                             "Total Pediatric Hospital<br />Beds Available", 
                             "Total Adult Hospital<br />Beds In-Use", "Total Pediatric Hospital<br />Beds In-Use",
                             "Total Hospital Beds<br />Used by COVID cases"]
                             
fig_2 = ff.create_table(df_2, colorscale=colorscale4)

fig_2.update_layout(
    {'margin':{'t':30}},
    title={
        "text":"Summary of hospital beds",
        "y":0.97,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    autosize=False,
    width=175*df_2.shape[1],
    height=60*df_2.shape[0]
)

fig_2.show()

In [30]:
# III
with conn.transaction():
    cur.execute(
        "SELECT facility_name, rating, "
        "total_adult_hospital_beds_occupied, "
        "total_pediatric_hospital_beds_occupied, "
        "total_icu_beds_occupied, "
        "inpatient_beds_occupied_covid "
        "FROM facility_information info "
        "JOIN quality_ratings rate "
        "ON info.facility_id = rate.facility_id "
        "JOIN facility_reports "
        "ON info.facility_id = hospital_pk "
        "ORDER BY rating DESC;"
        
    )

ratings = pd.DataFrame(
    cur.fetchall(), columns=["Hospital Name", "Hospital Rating",
                             "Total Adult Hospital Beds In-Use", "Total Pediatric Hospital Beds In-Use",
                             "Total ICU Beds In-Use", "Total Hospital Beds Used by COVID cases"]
)

In [31]:
ratings

,Hospital Name,Hospital Rating,Total Adult Hospital Beds In-Use,Total Pediatric Hospital Beds In-Use,Total ICU Beds In-Use,Total Hospital Beds Used by COVID cases
0,LBJ TROPICAL MEDICAL CENTER,None,59,10,5.4,0
1,LBJ TROPICAL MEDICAL CENTER,None,59,10,5.4,0
2,BIBB MEDICAL CENTER,None,7.3,0,0,0
3,HALE COUNTY HOSPITAL,None,7.4,0,0,0
4,HALE COUNTY HOSPITAL,None,7.4,0,0,0
...,...,...,...,...,...,...
68806,HCA FLORIDA LAWNWOOD HOSPITAL,1,325.9,20,64.6,12.7
68807,HCA FLORIDA LAWNWOOD HOSPITAL,1,325.9,20,64.6,12.7
68808,HCA FLORIDA LAWNWOOD HOSPITAL,1,325.9,20,64.6,12.7
68809,VICTOR VALLEY GLOBAL MEDICAL CENTER,1,41,0,5.3,None


In [32]:
# III - 2
with conn.transaction():
    cur.execute(
        "SELECT rating, "
        "SUM(total_adult_hospital_beds_occupied + total_pediatric_hospital_beds_occupied + "
        "total_icu_beds_occupied + inpatient_beds_occupied_covid) "
        "/ SUM(total_adult_hospital_beds + total_pediatric_hospital_beds + total_icu_beds) AS fraction, "
        "SUM(total_adult_hospital_beds_occupied + total_pediatric_hospital_beds_occupied + "
        "total_icu_beds_occupied + inpatient_beds_occupied_covid) AS total_used, "
        "SUM(total_adult_hospital_beds + total_pediatric_hospital_beds + total_icu_beds) AS total_beds "
        "FROM facility_information info "
        "JOIN quality_ratings rate "
        "ON info.facility_id = rate.facility_id "
        "JOIN facility_reports "
        "ON info.facility_id = hospital_pk "
        "GROUP BY rating "
        "ORDER BY rating DESC; "
        
    )

ratings_2 = pd.DataFrame(
    cur.fetchall(), columns=["Hospital Rating", "Fraction",
                             "Total Hospital Beds In-Use", "Total Hospital Beds"]
)

In [33]:
# III - 2 - table

df_2_2 = ratings_2

fig_2_2 = ff.create_table(df_2_2, colorscale=colorscale3)

fig_2_2.update_layout(
    {'margin':{'t':30}},
    title={
        "text":"Fraction of beds currently in use by hospital quality rating",
        "y":0.96,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    autosize=False,
    width=220*df_2_2.shape[1],
    height=40*df_2_2.shape[0]
)

fig_2_2.show()

In [34]:
# IV
with conn.transaction():
    # Extract the number of beds in use    
    cur.execute("SELECT report_date, SUM(total_occupied_0) "
                "AS total_occupied_all, "
                "SUM(inpatient_beds_occupied_covid) "
                "AS total_occupied_covid FROM "
                "(SELECT report_date, inpatient_beds_occupied_covid, "
                "total_adult_hospital_beds_occupied + "
                "total_pediatric_hospital_beds_occupied + "
                "total_icu_beds_occupied + "
                "inpatient_beds_occupied_covid AS total_occupied_0 "
                "FROM facility_reports) AS a "
                "GROUP BY report_date "
                "ORDER BY report_date;")
    data_used = pd.DataFrame(
        cur.fetchall(), columns=["report_date", 
                                 "all_used_beds",
                                 "covid_used_beds"]
        )


# Make the bar plot
fig_4_bar = go.Figure()
fig_4_bar.add_trace(go.Bar(
    x=data_used['report_date'],
    y=data_used['all_used_beds'],
    text=data_used['all_used_beds'],
    textposition='auto',
    name='All cases',
    marker_color='#41D3BD'
))
fig_4_bar.add_trace(go.Bar(
    x=data_used['report_date'],
    y=data_used['covid_used_beds'],
    text=data_used['covid_used_beds'],
    textposition='auto',
    name='COVID cases',
    marker_color='#379392'
))

fig_4_bar.update_layout(
    title={
        "text":"Number of beds in use of all cases and covid cases",
        "y":0.88,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    plot_bgcolor='#D8E6E7')

fig_4_bar.show()

In [35]:
# Make the pie chart
labels = ['All cases', 'COVID cases']

# Define color sets of paintings
colors1 = ['#88dba3', '#cff0da']
colors2 = ['#4ea1d3', '#d8e9ef']
colors3 = ['#6a60a9', '#dedcee']
colors4 = ['#fbd14b', '#fffcf0']
colors5 = ['#ee6e9f', '#fec9c9']

# Create subplots, using 'domain' type for pie charts
specs = [[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}], 
        [{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]]
fig_4_pie = make_subplots(rows=2, cols=3, specs=specs)

# Define pie charts
fig_4_pie.add_trace(go.Pie(labels=labels, values=[data_used["all_used_beds"][0],
                    data_used["covid_used_beds"][0]], name=str(data_used['report_date'][0]),
                     marker_colors=colors1), 1, 1)
fig_4_pie.add_trace(go.Pie(labels=labels, values=[data_used["all_used_beds"][1],
                    data_used["covid_used_beds"][1]], name=str(data_used['report_date'][1]),
                     marker_colors=colors2), 1, 2)
fig_4_pie.add_trace(go.Pie(labels=labels, values=[data_used["all_used_beds"][2],
                    data_used["covid_used_beds"][2]], name=str(data_used['report_date'][2]),
                     marker_colors=colors3), 1, 3)
fig_4_pie.add_trace(go.Pie(labels=labels, values=[data_used["all_used_beds"][3],
                    data_used["covid_used_beds"][3]], name=str(data_used['report_date'][3]),
                     marker_colors=colors4), 2, 1)
fig_4_pie.add_trace(go.Pie(labels=labels, values=[data_used["all_used_beds"][4],
                    data_used["covid_used_beds"][4]], name=str(data_used['report_date'][4]),
                     marker_colors=colors5), 2, 2)

# Tune layout and hover info
fig_4_pie.update_traces(hole=.4, hoverinfo='label+percent+name', textinfo='none')
fig_4_pie.update_layout(
    title={
        "text":"Number of beds in use of all cases and covid cases",
        "y":0.88,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    showlegend=False,
    annotations=[dict(text='9-23', x=0.125, y=0.825, font_size=13, showarrow=False),
                 dict(text='9-30', x=0.875, y=0.825, font_size=13, showarrow=False),
                 dict(text='10-07', x=0.5, y=0.825, font_size=13, showarrow=False),
                 dict(text='10-14', x=0.5, y=0.18, font_size=13, showarrow=False),
                 dict(text='10-21', x=0.12, y=0.18, font_size=13, showarrow=False)]
    )

fig_4_pie = go.Figure(fig_4_pie)
fig_4_pie.show()

In [36]:
# Additional I - COVID map - All weeks
# Make a new SAVEPOINT
with conn.transaction():    
    # Get the data for COVID map in each state
    cur.execute("SELECT state, SUM(total_covid_0) "
                "AS total_covid FROM "
                "(SELECT hospital_pk, adult_icu_patients_confirmed_covid "
                "+ inpatient_beds_occupied_covid AS total_covid_0 "
                "FROM facility_reports) AS a "
                "INNER JOIN "
                "(SELECT facility_id, state "
                "FROM facility_information) AS b "
                "ON a.hospital_pk = b.facility_id "
                "GROUP BY state; ")
    
    data_map1 = pd.DataFrame(cur.fetchall(), columns=["state", "beds"])

# Make the map
fig_map1 = go.Figure(data=go.Choropleth( 
    locations=data_map1['state'], 
    z = data_map1['beds'].astype(float), # Set fill color
    locationmode = 'USA-states', # Set the country 
    colorscale = 'reds', # Set the legend color
    colorbar_title = "Number", # Legend title
)) 
 
fig_map1.update_layout( 
    title={
        "text": "The number of COVID cases by state",
        "y":0.95,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    geo_scope='usa', # Set country
    autosize=True,
    margin=dict(t=30,b=25,l=0,r=0)
)


fig_map1.show()



In [37]:
# Additional I - COVID map - per week
with conn.transaction():
    # Get the data for COVID map in each state
    cur.execute("SELECT state, report_date, SUM(total_covid_0) "
                "AS total_covid FROM "
                "(SELECT hospital_pk, report_date, "
                "adult_icu_patients_confirmed_covid "
                "+ inpatient_beds_occupied_covid AS total_covid_0 "
                "FROM facility_reports "
                "GROUP BY report_date, hospital_pk) AS a "
                "INNER JOIN "
                "(SELECT facility_id, state "
                "FROM facility_information) AS b "
                "ON a.hospital_pk = b.facility_id "
                "GROUP BY state, report_date "
                "ORDER BY state;")
    
    data_map2 = pd.DataFrame(cur.fetchall(), columns=["state", "week", "beds"])


########## Note: need to change the date to input
data_map_tw = data_map2.iloc[np.where(
    data_map2["week"]==pd.Timestamp(week))[0],:]

# Make the map
fig_map2 = go.Figure(data=go.Choropleth( 
    locations=data_map_tw['state'], 
    z = data_map_tw['beds'].astype(float), # Set fill color
    locationmode = 'USA-states', # Set the country 
    colorscale = 'Reds', # Set the legend color
    colorbar_title = "Number", # Legend title
)) 

fig_map2.update_layout(
    title={
        ########## Note: need to change the date to input
        "text":"The number of COVID cases by state" + ": " + week,
        "y":0.95,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    geo_scope='usa', # Set country
    autosize=True,
    margin=dict(t=30,b=25,l=0,r=0)
)


/var/folders/vw/0kbg8m6n26sdbzft__pgcwc80000gn/T/ipykernel_18814/4137333525.py:23: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.



In [38]:
# Additional - II - State COVID increase
# Note: Change variables!
with conn.transaction():
    cur.execute(
        "SELECT tw.state, tw.covidcase - lw.covidcase AS coviddiff "
        "FROM ("
        # This weeks data
        "SELECT state, SUM(inpatient_beds_occupied_covid) as covidcase "
        "FROM facility_information "
        "JOIN facility_reports "
        "ON facility_id = hospital_pk "
        "WHERE report_date = %(week)s "
        "GROUP BY state) tw "
        "JOIN ("
        # Last weeks data
        "SELECT state, SUM(inpatient_beds_occupied_covid) as covidcase "
        "FROM facility_information "
        "JOIN facility_reports "
        "ON facility_id = hospital_pk "
        "WHERE report_date = %(lastweek)s "
        "GROUP BY state) lw "
        "ON tw.state = lw.state "
        "ORDER BY coviddiff DESC;",
        {
            "week": pd.Timestamp(week),
            "lastweek": pd.Timestamp(week) - pd.Timedelta(days=7)
        }
    )

statecovid = pd.DataFrame(
    cur.fetchall(), columns=["States", "Change in Covid Case"]
)

In [39]:
statecovid

,States,Change in Covid Case
0,GU,None
1,NY,122.9
2,NJ,37.8
3,ME,35.7
4,MA,33.5
5,NV,27.1
6,UT,15.0
7,IA,14.6
8,RI,10.7
9,MN,7.3


In [40]:
# Additional - II table
df_a_2 = statecovid

fig_a_2 = ff.create_table(df_a_2, colorscale=colorscale4)

fig_a_2.update_layout(
    {'margin':{'t':30}},
    title={
        "text":"State COVID cases changes",
        "y":0.995,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    autosize=False,
    width=240*df_a_2.shape[1],
    height=30*df_a_2.shape[0]
)

fig_a_2.show()

In [41]:
# Additional - III - Top Hospital COVID increase
# Note: Change variables!
with conn.transaction():
    cur.execute(
        "SELECT info.facility_name, tw.covidcase - lw.covidcase AS coviddiff, "
        "address, city, state, zipcode "
        "FROM ("
        # This weeks data
        "SELECT facility_id, inpatient_beds_occupied_covid as covidcase "
        "FROM facility_information "
        "JOIN facility_reports "
        "ON facility_id = hospital_pk "
        "WHERE report_date = %(week)s "
        ") tw "
        "JOIN ("
        # Last weeks data
        "SELECT facility_id, inpatient_beds_occupied_covid as covidcase "
        "FROM facility_information "
        "JOIN facility_reports "
        "ON facility_id = hospital_pk "
        "WHERE report_date = %(lastweek)s "
        ") lw "
        "ON tw.facility_id = lw.facility_id "
        "JOIN facility_information info "
        "ON info.facility_id = tw.facility_id "
        "WHERE tw.covidcase - lw.covidcase IS NOT NULL "
        "ORDER BY coviddiff DESC "
        "LIMIT 5",
        {
            "week": pd.Timestamp(week),
            "lastweek": pd.Timestamp(week) - pd.Timedelta(days=7)
        }
    )

hospitalcovid = pd.DataFrame(
    cur.fetchall(), columns=["Hospital Name", "Change in Covid Case",
                             "Address", "City", "State", "Zipcode"]
)

In [42]:
hospitalcovid

,Hospital Name,Change in Covid Case,Address,City,State,Zipcode
0,READING HOSPITAL,45.6,420 S 5TH AVENUE,WEST READING,PA,19611
1,HUNTINGTON HOSPITAL,19.3,270 PARK AVENUE,HUNTINGTON,NY,11743
2,STRONG MEMORIAL HOSPITAL,15.0,601 ELMWOOD AVE,ROCHESTER,NY,14642
3,MORRISTOWN MEDICAL CENTER,14.6,100 MADISON AVE,MORRISTOWN,NJ,7960
4,STATEN ISLAND UNIVERSITY HOSPITAL,14.0,475 SEAVIEW AVENUE,STATEN ISLAND,NY,10305


In [43]:
# Additional - III table
hospitalcovid_new = hospitalcovid

hospitalcovid_new["Hospital Name"] = [
    "CAROLINAS MEDICAL CENTER<br />/BEHAV HEALTH",
    "NEW YORK-PRESBYTERIAN HOSPITAL",
    "GEISINGER MEDICAL CENTER",
    "HOUSTON METHODIST HOSPITAL",
    "MAINE MEDICAL CENTER"]

df_a_3 = hospitalcovid_new

fig_a_3 = ff.create_table(df_a_3, colorscale=colorscale2)

fig_a_3.update_layout(
    {'margin':{'t':30}},
    title={
        "text":"Hospitals with largest changes in COVID cases",
        "y":0.97,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    autosize=False,
    width=240*df_a_3.shape[1],
    height=60*df_a_3.shape[0]
)

fig_a_3.show()

In [44]:
# Additional - IV - Hospital with no data
# Note: Change thisweek to the parameter of this week
with conn.transaction():
    cur.execute(
        "SELECT facility_name, MAX(report_date) "
        "FROM facility_information "
        "JOIN facility_reports ON "
        "facility_id = hospital_pk "
        "GROUP BY facility_id "
        "HAVING MAX(report_date) < %(week)s "
        "ORDER BY MAX(report_date) DESC;",
        {"week": pd.Timestamp(week)}
    )

nodata = pd.DataFrame(
    cur.fetchall(), columns=["Hospital Name", "Last reported Date"]
)

In [45]:
nodata

,Hospital Name,Last reported Date
0,BLESSING HEALTH KEOKUK,2022-09-23


In [67]:
# Additional - IV table
df_a_4 = nodata

fig_a_4 = ff.create_table(df_a_4, colorscale=colorscale1)

fig_a_4.update_layout(
    {'margin':{'t':30}},
    title={
        "text":"Hospital with no reported data",
        "y":0.93,
        "x":0.5,
        "xanchor":"center",
        "yanchor":"top"
    },
    autosize=True
)

fig_a_4.show()